# Verify Preprocessed Scenes

Sanity check that scene conversion is correct and random cropping works.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import random
import json

# Path to preprocessed scenes
SCENE_DIR = Path('../data/ai4arctic_hugging face/npy_scenes')

# Class names
CLASS_NAMES = [
    'OpenWater',
    'NewIce',
    'YoungIce',
    'FirstYearIce',
    'MultiYearIce',
    'GlacialIce',
]

# Colors for each class
COLORS = [
    [0, 0, 139],      # OpenWater - dark blue
    [173, 216, 230],  # NewIce - light blue
    [144, 238, 144],  # YoungIce - light green
    [255, 255, 0],    # FirstYearIce - yellow
    [255, 165, 0],    # MultiYearIce - orange
    [255, 255, 255],  # GlacialIce - white
]
COLORS = np.array(COLORS) / 255.0

In [ ]:
# Check preprocessing output
train_scenes = list((SCENE_DIR / 'train').glob('*.npz'))
val_scenes = list((SCENE_DIR / 'val').glob('*.npz'))

print(f'Train scenes: {len(train_scenes)}')
print(f'Val scenes: {len(val_scenes)}')

# Load metadata
if (SCENE_DIR / 'metadata.json').exists():
    with open(SCENE_DIR / 'metadata.json') as f:
        metadata = json.load(f)
    print(f"\nMetadata:")
    print(f"  Version: {metadata['version']}")
    print(f"  Classes: {metadata['num_classes']}")

# Load normalization stats
if (SCENE_DIR / 'normalization_stats.json').exists():
    with open(SCENE_DIR / 'normalization_stats.json') as f:
        stats = json.load(f)
    print(f"\nNormalization stats:")
    for i, name in enumerate(stats['channel_names']):
        print(f"  {name}: mean={stats['mean'][i]:.4f}, std={stats['std'][i]:.4f}")

In [ ]:
def load_scene(path):
    """Load a scene and return image, label."""
    data = np.load(path)
    image = data['image']  # [3, H, W]
    label = data['label']  # [H, W]
    return image, label

def label_to_rgb(label):
    """Convert label to RGB image for visualization."""
    H, W = label.shape
    rgb = np.zeros((H, W, 3))
    
    for cls_idx, color in enumerate(COLORS):
        mask = label == cls_idx
        rgb[mask] = color
    
    # Mark ignore pixels (255) as gray
    ignore_mask = label == 255
    rgb[ignore_mask] = [0.5, 0.5, 0.5]
    
    return rgb

def visualize_scene(path, crop_size=256):
    """Visualize a full scene with a random crop highlighted."""
    image, label = load_scene(path)
    H, W = label.shape
    
    # Pick a random crop location
    r = random.randint(0, max(0, H - crop_size))
    c = random.randint(0, max(0, W - crop_size))
    
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    
    # Top row: Full scene
    axes[0, 0].imshow(image[0], cmap='gray', vmin=np.nanpercentile(image[0], 1), vmax=np.nanpercentile(image[0], 99))
    axes[0, 0].set_title(f'Full Scene HH [{image.shape[1]}x{image.shape[2]}]')
    axes[0, 0].add_patch(plt.Rectangle((c, r), crop_size, crop_size, fill=False, edgecolor='red', linewidth=2))
    
    axes[0, 1].imshow(image[1], cmap='gray', vmin=np.nanpercentile(image[1], 1), vmax=np.nanpercentile(image[1], 99))
    axes[0, 1].set_title('Full Scene HV')
    axes[0, 1].add_patch(plt.Rectangle((c, r), crop_size, crop_size, fill=False, edgecolor='red', linewidth=2))
    
    axes[0, 2].imshow(image[2], cmap='viridis')
    axes[0, 2].set_title('Full Scene Incidence Angle')
    axes[0, 2].add_patch(plt.Rectangle((c, r), crop_size, crop_size, fill=False, edgecolor='red', linewidth=2))
    
    label_rgb = label_to_rgb(label)
    axes[0, 3].imshow(label_rgb)
    axes[0, 3].set_title('Full Scene SOD Label')
    axes[0, 3].add_patch(plt.Rectangle((c, r), crop_size, crop_size, fill=False, edgecolor='red', linewidth=2))
    
    # Bottom row: Crop (what training sees)
    crop_img = image[:, r:r+crop_size, c:c+crop_size]
    crop_lbl = label[r:r+crop_size, c:c+crop_size]
    
    axes[1, 0].imshow(crop_img[0], cmap='gray')
    axes[1, 0].set_title(f'Crop HH [{crop_size}x{crop_size}]')
    
    axes[1, 1].imshow(crop_img[1], cmap='gray')
    axes[1, 1].set_title('Crop HV')
    
    axes[1, 2].imshow(crop_img[2], cmap='viridis')
    axes[1, 2].set_title('Crop Incidence Angle')
    
    crop_rgb = label_to_rgb(crop_lbl)
    axes[1, 3].imshow(crop_rgb)
    axes[1, 3].set_title(f'Crop Label (unique: {np.unique(crop_lbl).tolist()})')
    
    for ax in axes.flat:
        ax.axis('off')
    
    plt.suptitle(path.name, fontsize=12)
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize 3 random train scenes
print('=== TRAIN SCENES ===')
for scene_path in random.sample(train_scenes, min(3, len(train_scenes))):
    visualize_scene(scene_path)

In [ ]:
# Visualize 2 random val scenes
print('=== VAL SCENES ===')
for scene_path in random.sample(val_scenes, min(2, len(val_scenes))):
    visualize_scene(scene_path)

In [ ]:
# Check scene sizes and class distribution
print('\n=== SCENE STATISTICS ===')

heights, widths = [], []
class_pixels = {i: 0 for i in range(6)}
ignore_pixels = 0
total_pixels = 0

# Sample scenes for distribution
sample_scenes = random.sample(train_scenes, min(50, len(train_scenes)))

for path in sample_scenes:
    _, label = load_scene(path)
    heights.append(label.shape[0])
    widths.append(label.shape[1])
    
    for cls_idx in range(6):
        class_pixels[cls_idx] += (label == cls_idx).sum()
    ignore_pixels += (label == 255).sum()
    total_pixels += label.size

print(f'\nScene dimensions (from {len(sample_scenes)} scenes):')
print(f'  Height: min={min(heights)}, max={max(heights)}, mean={np.mean(heights):.0f}')
print(f'  Width:  min={min(widths)}, max={max(widths)}, mean={np.mean(widths):.0f}')

print(f'\nPixel distribution:')
for cls_idx, count in class_pixels.items():
    pct = count / total_pixels * 100
    print(f'  {CLASS_NAMES[cls_idx]}: {pct:.2f}%')
print(f'  Ignore (255): {ignore_pixels / total_pixels * 100:.2f}%')

In [ ]:
# Legend
fig, ax = plt.subplots(figsize=(10, 1))
for i, (name, color) in enumerate(zip(CLASS_NAMES, COLORS)):
    ax.add_patch(plt.Rectangle((i, 0), 1, 1, color=color, edgecolor='black'))
    ax.text(i + 0.5, 0.5, name, ha='center', va='center', fontsize=9,
            color='black' if i != 5 else 'gray')  # GlacialIce is white, use gray text
ax.add_patch(plt.Rectangle((6, 0), 1, 1, color=[0.5, 0.5, 0.5], edgecolor='black'))
ax.text(6.5, 0.5, 'Ignore', ha='center', va='center', fontsize=9)
ax.set_xlim(0, 7)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title('Class Legend')
plt.tight_layout()
plt.show()

In [ ]:
# Test the dataloader
import sys
sys.path.insert(0, '../src')
from dataset import get_dataloaders

train_loader, val_loader = get_dataloaders(
    data_dir='../data/ai4arctic_hugging face',
    batch_size=4,
    num_workers=0,
    crop_size=256,
)

print(f'Train batches: {len(train_loader)}')
print(f'Val batches: {len(val_loader)}')

In [ ]:
# Visualize a batch from dataloader
batch = next(iter(train_loader))
images = batch['image']  # [B, 3, 256, 256]
labels = batch['label']  # [B, 256, 256]

print(f'Batch image shape: {images.shape}')
print(f'Batch label shape: {labels.shape}')
print(f'Image range: [{images.min():.2f}, {images.max():.2f}]')
print(f'Label unique values: {torch.unique(labels).tolist()}')

fig, axes = plt.subplots(4, 4, figsize=(16, 16))
for i in range(4):
    # HH
    axes[i, 0].imshow(images[i, 0].numpy(), cmap='gray')
    axes[i, 0].set_title(f'Sample {i} - HH')
    axes[i, 0].axis('off')
    
    # HV
    axes[i, 1].imshow(images[i, 1].numpy(), cmap='gray')
    axes[i, 1].set_title(f'Sample {i} - HV')
    axes[i, 1].axis('off')
    
    # Incidence
    axes[i, 2].imshow(images[i, 2].numpy(), cmap='viridis')
    axes[i, 2].set_title(f'Sample {i} - Incidence')
    axes[i, 2].axis('off')
    
    # Label (convert -100 back to 255 for visualization)
    lbl = labels[i].numpy().copy()
    lbl[lbl == -100] = 255
    axes[i, 3].imshow(label_to_rgb(lbl.astype(np.uint8)))
    axes[i, 3].set_title(f'Sample {i} - Label')
    axes[i, 3].axis('off')

plt.suptitle('Batch from DataLoader (normalized, augmented)', fontsize=14)
plt.tight_layout()
plt.show()